In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("level1software").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from Zillow


In [ ]:
# Read in fed rate data


In [ ]:
# Limit Zillow data to past 10 years

In [ ]:
# Limit fed rate data to past 10 years

In [ ]:
# Add fed rate data to Zillow data

In [ ]:
# Show number of rows

In [ ]:
# Remove duplicates and NAN values

In [ ]:
# Show number of rows

In [ ]:
# Set up Target as most recent price (y) and Features as rest of dataframe (X)

In [ ]:
# Linear Regression Model

In [ ]:
# Analysis